In [6]:
from src.db.database import session_scope
from src.db import models

with session_scope() as session:
    res = session.query(models.Firms.sector).distinct().all()
    res = [r[0] for r in res]

res

['Electric Utilities',
 'Communications Equipment',
 'Life Sciences Tools & Services',
 'Health Care REITs',
 'Electronic Equipment & Instruments',
 'Application Software',
 'Asset Management & Custody Banks',
 'IT Consulting & Other Services',
 'Casinos & Gaming',
 'Health Care Distributors',
 'Air Freight & Logistics',
 'Automobile Manufacturers',
 'Diversified Banks',
 'Specialty Chemicals',
 'Publishing',
 'Integrated Telecommunication Services',
 'Packaged Foods & Meats',
 'Construction Machinery & Heavy Transportation Equipment',
 'Consumer Finance',
 'Passenger Airlines',
 'Financial Exchanges & Data',
 'Health Care Equipment',
 'Oil & Gas Exploration & Production',
 'Restaurants',
 'Life & Health Insurance',
 'Industrial Machinery & Supplies & Components',
 'Insurance Brokers',
 'Homebuilding',
 'Heavy Electrical Equipment',
 'Biotechnology',
 'Rail Transportation',
 'Hotel & Resort REITs',
 'Multi-Utilities',
 'Oil & Gas Refining & Marketing',
 'Tobacco',
 'Systems Software',


# Llama-Index Text-To-SQL Retrieval Agent
### Thoughts:
- Too inconsistent in its performance
- Easily makes up facts in the absence of results
- Isn't really able to grasp the full context of the data structure and meaning
- Underlying functionality difficult to modify, particularly the prompt template for the text-to-sql process prior to response synthesis.

In [7]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import pandas as pd

from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex, PromptTemplate

from src.db.database import engine
from src.db import models


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


llm = OpenAI(temperature=0.1, model="gpt-4o-mini", api_key=OPENAI_API_KEY)

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = [
    (SQLTableSchema(table_name=table.__tablename__, context_str=table.__context_str__)) 
    for table in models.__dict__.values() if hasattr(table, '__tablename__')
]

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results. \
    You must ensure your response is completely factual.\n"
    "<query>{query_str}</query>\n"
    "<sql>{sql_query}</sql>\n"
    "<sql response>SQL Response: {context_str}</sql response>\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1),
    response_synthesis_prompt=response_synthesis_prompt,
)

# query = "What are the fields in the meetings table and what do they represent contextually?"
# query = "Using just your provided system messaging and without using SQL, \
#     What are the fields in the meetings table and what do they represent contextually?"
# query = "What is the name of the firm that has the most meetings and how many meets do they have?"
# query = "Can you show me the first 5 rows of meetings?"
query = "Fetch the first 5 meetings and their content which have a firm attended that are in the IT Consulting & Other Services sector."
response = query_engine.query(query)

print("SQL Query:")
print("```\n" + response.metadata["sql_query"] + "\n```")
print("Response:")
display(Markdown(f"<b>{response}</b>"))
if "result" in response.metadata:
    display(pd.DataFrame(response.metadata["result"], columns=response.metadata["col_keys"]))

SQL Query:
```
SELECT m.meeting_id, m.title, m.content
FROM meetings m
JOIN firms f ON m.firm_attended_id = f.firm_id
WHERE f.sector = 'IT Consulting & Other Services'
ORDER BY m.date
LIMIT 5;
```
Response:


<b>The first 5 meetings attended by firms in the IT Consulting & Other Services sector are as follows:
1. Meeting with Accenture: Discussed potential synergies, tech innovations, data analytics, regulatory changes, and improved communication channels.
2. Call with Cognizant: Discussed collaboration opportunities, growth in digital services, key sectors for investment, and regulatory challenges.
3. Call with Accenture: Discussed Verizon's market performance, investment in 5G technology, competition, customer retention strategies, and potential for M&A.
4. Call with Cognizant: Discussed Incyte's pipeline developments, GE HealthCare's market position, Entergy's regulatory challenges, and Gilead Sciences' acquisitions.
5. Email with Cognizant: Discussion on potential collaborations with IQVIA, Yum! Brands, Host Hotels & Resorts, and Cummins.</b>

,meeting_id,title,content
0,b1df9bfc-febd-44d2-9cf4-ae5e905953d5,Meeting with Accenture,- Discussed potential synergies between our fi...
1,93ee4cf5-ca97-48ce-ac42-e9b01e83c605,Call with Cognizant,- Discussed potential collaboration opportunit...
2,6539b388-0b71-4ed4-b792-a3088d7496c6,Call with Accenture,- Discussed Verizon's recent market performanc...
3,18d357db-b908-4360-8054-3c16bccbba8f,Call with Cognizant,- Discussed Incyte's recent pipeline developme...
4,8cb7004e-07f8-4180-b1cd-acf71a3a6a2b,Email with Cognizant,Subject: Discussion on Potential Collaboration...


# Custom Simplified Implementation
- Much slower
- Has chain of thought reasoning with verbosity
- Still has issues constructing queries
- Need to consider how the information is presented back to the User in a memory-friendly way
    - Can return just beam_ids as part of the retrieval?
        - This can be added to the user's 'meetings in-focus' view?
    - Can return as markdown (BIG CONTEXT ISSUE)

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from llama_index.llms.openai import OpenAI

from src.db.database import session_scope
from src.db import models
from src.rag.sql_retriever import SQLAgent

load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(temperature=0.1, model="gpt-4o-mini", api_key=OPENAI_API_KEY)

agent = SQLAgent(llm, "src/db/models.py", verbose=True)

# query = "I need all meetings between 2022-01-01 and 2023-01-01 where the firms that attended are in the Energy sector."
query = "Return the beam_ids of all meetings between 2022-01-01 and 2023-01-01 where the firms that attended are in the IT Consulting & Other Services sector."

with session_scope() as session:
    response = agent.complete(session, query)

response_md = response.to_markdown(index=True)

print(response_md)

CHAIN OF THOUGHTS:
Thoughts: I need to retrieve the beam_ids of meetings that occurred between 2022-01-01 and 2023-01-01. I also need to filter these meetings based on the sector of the firms that attended, specifically looking for firms in the 'IT Consulting & Other Services' sector.
Outcome: I will need to join the meetings table with the firms table to filter by sector and select the beam_id from the meetings table. 

Thoughts: The meetings table has a foreign key to the firms table through the firm_attended_id. I will join the meetings table with the firms table using this foreign key.
Outcome: I will add a join between the meetings and firms tables on the firm_attended_id and firm_id. 

Thoughts: I need to filter the results to only include meetings that occurred between the specified dates and where the firm's sector matches 'IT Consulting & Other Services'.
Outcome: I will add a filter for the meeting date and the firm's sector. 

Thoughts: Finally, I need to ensure that the res

# Some additional tests

In [2]:
with session_scope() as session:
    res = (
        session.query(models.Meetings)
        .filter(models.Meetings.date >= "2025-01-15", models.Meetings.date <= "2025-02-15")
        .all()
    )
    for meeting in res:
        meeting.employees
    

res = [x for y in [[str(y.employee_id) for y in x.employees] for x in res] for x in y]
user_id = pd.Series(res).value_counts().sort_values(ascending=False).index[0]

In [3]:
with session_scope() as session:
    res = (
        session.query(models.Meetings)
        .filter(
            models.Meetings.date >= "2025-01-15", 
            models.Meetings.date <= "2025-02-15",
            models.Meetings.employees.any(models.Employees.employee_id == user_id)
        )
        .all()
    )

len(res)

2

In [4]:
query = "Give me a summary of meetings I have attended in the last month (employee_id: {}, current date: {}).".format(user_id, "2025-02-15")

with session_scope() as session:
    response = agent.complete(session, query)

response_md = response.to_markdown(index=True)
print(response_md)

CHAIN OF THOUGHTS:
Thoughts: I need to retrieve meetings attended by a specific employee within the last month. The employee's ID is provided, and the current date is given as 2025-02-15. I will need to filter meetings based on the date range from 2025-01-15 to 2025-02-15.
Outcome: I will filter the meetings based on the date being greater than or equal to 2025-01-15 and less than or equal to 2025-02-15. 

Thoughts: The meetings attended by employees are stored in the 'employee_meetings' association table. I need to join this table with the 'meetings' table to get the meeting details.
Outcome: I will join the 'employee_meetings' table with the 'meetings' table on the meeting_id. 

Thoughts: I need to select the relevant fields from the meetings table, including the beam_id, title, content, date, and created_at. I will also ensure to order the results by date in ascending order.
Outcome: I will select the fields 'meetings.beam_id', 'meetings.title', 'meetings.content', 'meetings.date', 

# Expand into Query -> Answer

In [10]:
from typing import Optional
from textwrap import dedent
from pydantic import BaseModel
from tenacity import retry, stop_after_attempt
from llama_index.core import PromptTemplate

prompt_template = PromptTemplate(
    dedent(
        """\
        You are a ChatBot built by Harvery's & Co, an investment bank. \
        Harvery's & Co specialise in investment banking, mergers and acquisitions, and asset management. \
        Your task is to answer employee's queries relating to company meeting notes held in a database. \
        Given a query by a user, a SQL AI Agent will try to find the relevant data in the database to answer the query. \
        Your task is to:\n
            1. read the user's query and the returned data from the database.\n
            2. analyse the retrieved data and how it might relate to the user's query.\n
            3. write a response back to the user to answer their query.\n

        ## IMPORTANT
        - **Your output must use the structured output format provided.**\n
        - **If the retrieved data does not answer the user's query, you must tell the user this and ask for more context to help you answer their query.**\n
        - **You must not make information up that does not exist in the database.**\n
        - **When referencing or citing meeting data from the database, you MUST provide the meetings.beam_id of the meeting encased in xml tags <ref>beam_id</ref>**\n
        - **You must stylise your response in markdown to make it easier to read by humans.**\n\n

        # User Query:\n
        <query>{query}</query>\n\n

        # Retrieved Data:\n
        <data>{data}</data>\n\n
        """
    )
)

query_writer_template = PromptTemplate(
    dedent(
        """\
        You are a ChatBot built by Harvery's & Co, an investment bank. \
        Harvery's & Co specialise in investment banking, mergers and acquisitions, and asset management. \
        Your task is to answer employee's queries relating to company meeting notes held in a database. \
        You must accomplish this task by cooperating with a SQL AI Agent that can retrieve data from the database. \
        Given a query by a user, you must instruct the SQL AI Agent using ONLY natural language to retrieve the relevant data from the database.\n\n
        
        ## IMPORTANT
        - **It is important that you provide clear and concise instructions to the SQL AI Agent, including any dates, ids, or personal details the \
        user has mentioned that is relevant to their query.**\n
        - **You must not write SQL queries yourself, ONLY provide natural language instructions to the SQL AI Agent.**\n

        # User Query:\n
        <query>{query}</query>\n\n
        """
    )
)

class Step(BaseModel):
    """
    Use this class to think about the user's query and the retrieved data and how it might relate to the user's query.
    You can also use it to store beam_ids you want to reference in your response.

    Attributes:
    - thought: (str) - Your thoughts on how the retrieved data might relate to the user's query.
    - beam_ids: Optional[list[str]] - A list of beam_ids of the meetings that were used to answer the user's query.
    - conclusion: (str) - Your conclusion on how the retrieved data relates to the user's query.
    """
    thought: str
    beam_ids: Optional[list[str]] = []
    conclusion: str


class Response(BaseModel):
    """
    Use this class to structure your response back to the user.

    Attributes:
    - steps: list[Step] - A list of Step objects.
    - response: (str) - Your response back to the user.
    - beam_ids: Optional[list[str]] - An exhaustive list of the beam_ids of the meetings that were used to answer the user's query.
    """
    steps: list[Step]
    response: str
    beam_ids: Optional[list[str]] = []


class SQLQnA:
    def __init__(
            self, 
            llm: OpenAI, 
            agent: SQLAgent, 
            prompt_template: PromptTemplate,
            query_writer_template: PromptTemplate,
            output_format: Response = Response,
            verbose: bool = False,
            _query_write_max_tokens: int = 250,
            _response_max_tokens: int = 4000,
        ):
        self.agent = agent
        self.llm = llm.as_structured_llm(output_format)
        self.prompt_template = prompt_template
        self.query_writer_template = query_writer_template
        self._query_write_max_tokens = _query_write_max_tokens
        self._response_max_tokens = _response_max_tokens
        self._verbose = verbose

    def _query_db(self, query: str) -> pd.DataFrame:
        with session_scope() as session:
            response = self.agent.complete(session, query)
        return response

    def _get_response_md(self, query: str) -> str:
        response = self._query_db(query)
        if isinstance(response, str):
            return response
        return response.to_markdown(index=True)
    
    @retry(stop=stop_after_attempt(3))
    def _invoke_llm(self, prompt_template: PromptTemplate, max_tokens: int, **kwargs) -> Response:
        return self.llm.complete(
            prompt_template.format(**kwargs),
            max_tokens=max_tokens
        ).raw
    
    def complete(self, query: str) -> Response:
        ai_query = self._invoke_llm(self.query_writer_template, self._query_write_max_tokens, query=query)
        if self._verbose:
            print("AI QUERY:")
            print(ai_query.response)
        data = self._get_response_md(ai_query.response)
        if self._verbose:
            print("RETURNED DATA:")
            print(data)
        response = self._invoke_llm(self.prompt_template, self._response_max_tokens, query=query, data=data)
        return response
    

query = "Give me a summary of meetings I have attended in the last month (employee_id: {}, current date: {}).".format(user_id, "2025-02-15")

qna_agent = SQLQnA(
    llm=OpenAI(temperature=0.1, model="gpt-4o-mini", api_key=OPENAI_API_KEY),
    agent=agent,
    prompt_template=prompt_template,
    query_writer_template=query_writer_template,
    output_format=Response,
    verbose=True
)

response = qna_agent.complete(query)
print("RESPONSE:")
print(response.response)
print(response.beam_ids)

AI QUERY:
Please retrieve a summary of all meetings attended by the employee with ID 'ed69b4cb-7104-427c-bf5d-fc3c4083138e' from January 15, 2025, to February 15, 2025.
CHAIN OF THOUGHTS:
Thoughts: I need to retrieve meetings attended by a specific employee identified by their ID. The employee's ID is 'ed69b4cb-7104-427c-bf5d-fc3c4083138e'.
Outcome: I will filter the meetings based on the employee's ID. 

Thoughts: The meetings should be filtered by date, specifically from January 15, 2025, to February 15, 2025.
Outcome: I will add a date filter to the query. 

Thoughts: I need to include the beam_id field in the results, along with other relevant meeting details.
Outcome: I will select the beam_id, title, content, date, and created_at fields from the meetings table. 

Thoughts: I need to join the employee_meetings table to link employees to their attended meetings.
Outcome: I will include a join between the meetings and employee_meetings tables. 

Thoughts: Finally, I need to order th